In [1]:
from datetime import datetime
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import UFC_Scrape_Library as LIB

In [2]:
urls_yearly_events = []
currentyear = datetime.now().year

for year in range(1994, currentyear + 1):
    newurl = 'https://mmadecisions.com/decisions-by-event/' + str(year) + '/'
    urls_yearly_events.append(newurl)

urls_yearly_events

['https://mmadecisions.com/decisions-by-event/1994/',
 'https://mmadecisions.com/decisions-by-event/1995/',
 'https://mmadecisions.com/decisions-by-event/1996/',
 'https://mmadecisions.com/decisions-by-event/1997/',
 'https://mmadecisions.com/decisions-by-event/1998/',
 'https://mmadecisions.com/decisions-by-event/1999/',
 'https://mmadecisions.com/decisions-by-event/2000/',
 'https://mmadecisions.com/decisions-by-event/2001/',
 'https://mmadecisions.com/decisions-by-event/2002/',
 'https://mmadecisions.com/decisions-by-event/2003/',
 'https://mmadecisions.com/decisions-by-event/2004/',
 'https://mmadecisions.com/decisions-by-event/2005/',
 'https://mmadecisions.com/decisions-by-event/2006/',
 'https://mmadecisions.com/decisions-by-event/2007/',
 'https://mmadecisions.com/decisions-by-event/2008/',
 'https://mmadecisions.com/decisions-by-event/2009/',
 'https://mmadecisions.com/decisions-by-event/2010/',
 'https://mmadecisions.com/decisions-by-event/2011/',
 'https://mmadecisions.com/d

In [3]:
df_event = pd.read_csv('MMA_Events.csv')
urls_events = df_event['url'].tolist()
    
for url in urls_yearly_events:

    if url in urls_events:
        continue

    df_event_new = LIB.scrape_event_data(url)

    df_event_all = pd.concat([df_event, df_event_new], ignore_index=True)

# Sort descending by date
df_event_all['Date'] = pd.to_datetime(df_event_all['Date']).dt.date
df_event_all = df_event_all.sort_values(by='Date', ascending=False)

df_event_all.head(10)

,Date,Event,NumFights,url
1514,2025-06-07,UFC 316: Dvalishvili vs. O''Malley 2,1,https://mmadecisions.com/event/1550/UFC-316-Dv...
1,2025-04-26,UFC on ESPN 66: Machado Garry vs. Prates,4,https://mmadecisions.com/event/1543/UFC-on-ESP...
0,2025-04-26,CW 188: Cage Warriors 188,8,https://mmadecisions.com/event/1542/CW-188-Cag...
2,2025-04-19,CW 187: Glasgow,3,https://mmadecisions.com/event/1541/CW-187-Gla...
3,2025-04-18,PFL 3: 2025 Season,2,https://mmadecisions.com/event/1540/PFL-3-2025...
4,2025-04-12,UFC 314: Volkanovski vs. Lopes,5,https://mmadecisions.com/event/1538/UFC-314-Vo...
5,2025-04-11,PFL 2: 2025 Season,3,https://mmadecisions.com/event/1539/PFL-2-2025...
6,2025-04-05,UFC on ESPN 65: Emmett vs. Murphy,8,https://mmadecisions.com/event/1536/UFC-on-ESP...
7,2025-04-03,PFL 1: 2025 Season,2,https://mmadecisions.com/event/1537/PFL-1-2025...
8,2025-03-29,UFC on ESPN 64: Moreno vs. Erceg,7,https://mmadecisions.com/event/1535/UFC-on-ESP...


In [4]:
# Save to MMA_Events csv
df_event_all.to_csv('MMA_Events.csv', index=False)

In [5]:
all_fight_data = []
df_decision = pd.read_csv('MMA_Decisions.csv')
urls_decisions = df_decision['url'].tolist()

for url in df_event_all['url']:

  if url in urls_decisions:
      continue

  try:
      df_decision_new = LIB.scrape_fight_data(url)
      df_decision = pd.concat([df_decision, df_decision_new], ignore_index=True)
  except Exception as e:
    continue
    
# Sort descending by date
df_decision = df_decision.sort_values(by='url', ascending=False)

df_decision.head(10)


,Event,Location,Venue,url
9922,CW 104: Shore vs. Malone,"Cardiff, Wales",Viola Arena,https://mmadecisions.com/decision/9997/Mason-J...
2899,CW 104: Shore vs. Malone,"Cardiff, Wales",Viola Arena,https://mmadecisions.com/decision/9997/Mason-J...
2900,CW 104: Shore vs. Malone,"Cardiff, Wales",Viola Arena,https://mmadecisions.com/decision/9994/Kris-Ed...
9923,CW 104: Shore vs. Malone,"Cardiff, Wales",Viola Arena,https://mmadecisions.com/decision/9994/Kris-Ed...
9924,CW 104: Shore vs. Malone,"Cardiff, Wales",Viola Arena,https://mmadecisions.com/decision/9993/Josh-Re...
2901,CW 104: Shore vs. Malone,"Cardiff, Wales",Viola Arena,https://mmadecisions.com/decision/9993/Josh-Re...
9931,Bellator 220: MacDonald vs. Fitch,"San Jose, California, USA",SAP Center,https://mmadecisions.com/decision/9990/Benson-...
2896,Bellator 220: MacDonald vs. Fitch,"San Jose, California, USA",SAP Center,https://mmadecisions.com/decision/9990/Benson-...
9930,Bellator 220: MacDonald vs. Fitch,"San Jose, California, USA",SAP Center,https://mmadecisions.com/decision/9989/Rory-Ma...
2895,Bellator 220: MacDonald vs. Fitch,"San Jose, California, USA",SAP Center,https://mmadecisions.com/decision/9989/Rory-Ma...


In [6]:
# Save to MMA_Scorecards csv
df_decision.to_csv('MMA_Decisions.csv', index=False)

In [7]:
error_urls = []
df_scorecards = pd.read_csv('MMA_Scorecards.csv')
urls_scorecards = df_decision['url'].tolist()

for url in df_decision['url']:

  if url in urls_scorecards:
      continue

  try:
      df_scorecards_new = LIB.scrape_scorecard(url)
      df_scorecards = pd.concat([df_scorecards, df_scorecards_new], ignore_index=True)
  except Exception:
    error_urls.append(url)
    continue
    
# Sort descending by date
df_scorecards = df_scorecards.sort_values(by='url', ascending=False)

df_scorecards.head(10)


,Fight,Judge,Round,FighterOne,FighterTwo,ScoreOne,ScoreTwo,url
4342,Mason Jones vs Donovan Desmae,Ben Cartlidge,1,Mason Jones,Donovan Desmae,10,9,https://mmadecisions.com/decision/9997/Mason-J...
18073,Mason Jones vs Donovan Desmae,David Lethaby,2,Mason Jones,Donovan Desmae,10,9,https://mmadecisions.com/decision/9997/Mason-J...
18075,Mason Jones vs Donovan Desmae,David Lethaby,1,Mason Jones,Donovan Desmae,10,9,https://mmadecisions.com/decision/9997/Mason-J...
42918,Mason Jones vs Donovan Desmae,Mark Collett,1,Mason Jones,Donovan Desmae,9,10,https://mmadecisions.com/decision/9997/Mason-J...
4306,Mason Jones vs Donovan Desmae,Ben Cartlidge,2,Mason Jones,Donovan Desmae,10,9,https://mmadecisions.com/decision/9997/Mason-J...
19537,Mason Jones vs Donovan Desmae,David Lethaby,3,Mason Jones,Donovan Desmae,10,9,https://mmadecisions.com/decision/9997/Mason-J...
4265,Mason Jones vs Donovan Desmae,Ben Cartlidge,3,Mason Jones,Donovan Desmae,10,9,https://mmadecisions.com/decision/9997/Mason-J...
42894,Mason Jones vs Donovan Desmae,Mark Collett,3,Mason Jones,Donovan Desmae,10,9,https://mmadecisions.com/decision/9997/Mason-J...
42936,Mason Jones vs Donovan Desmae,Mark Collett,2,Mason Jones,Donovan Desmae,10,9,https://mmadecisions.com/decision/9997/Mason-J...
18072,Kris Edwards vs Josh Abraham,David Lethaby,1,Kris Edwards,Josh Abraham,10,9,https://mmadecisions.com/decision/9994/Kris-Ed...


In [8]:
# Save to MMA_Scorecards csv
df_scorecards.to_csv('MMA_Scorecards.csv', index=False)